# 🐄 Cow Video Extraction - Google Colab Edition

Bu notebook, **Cow Extraction Project**'i Google Colab üzerinde çalıştırmak için hazırlanmıştır. 
Güçlü bulut GPU'ları (T4, V100, A100) kullanarak işlemleri hızlandırabilir ve `yolov8x-seg` gibi büyük modelleri kullanabilirsiniz.

### Adımlar:
1. GPU Kontrolü
2. Google Drive Bağlantısı
3. Kurulum
4. Ayarların Yapılması
5. İşlemin Başlatılması

In [ ]:
# 1. GPU Kontrolü
!nvidia-smi

In [ ]:
# 2. Google Drive Bağlantısı
from google.colab import drive
drive.mount('/content/drive')

print("Drive başarıyla bağlandı.")

In [ ]:
# 3. Proje Dosyalarına Erişim
import os

# Proje klasörünüzün Drive içindeki yolu. 
# Eğer projeyi Drive'a yüklediyseniz burayı düzenleyin.
PROJECT_PATH = '/content/drive/MyDrive/CowExtractionFromVideos' 

if os.path.exists(PROJECT_PATH):
    %cd {PROJECT_PATH}
    print(f"Çalışma dizini değiştirildi: {os.getcwd()}")
else:
    print(f"HATA: {PROJECT_PATH} bulunamadı. Lütfen yolu kontrol edin.")

In [ ]:
# 4. Gereksinimlerin Yüklenmesi
!pip install -r requirements.txt
!pip install ultralytics  # Emin olmak için tekrar yüklüyoruz

In [ ]:
# 5. Konfigürasyon Dosyasını Düzenleme (Opsiyonel ama Önemli)
# Colab ortamına özel yolları ayarlamak için mevcut settings.py dosyasını güncelliyoruz.

settings_content = """
import os

# Base directory
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

# --- COLAB PATH AYARLARI ---
# Drive üzerindeki video klasör yolları
INPUT_VIDEOS_DIR = '/content/drive/MyDrive/CowExtractionFromVideos/input_videos'
OUTPUT_VIDEOS_DIR = '/content/drive/MyDrive/CowExtractionFromVideos/output_cows'
SINGLE_COW_VIDEOS_DIR = '/content/drive/MyDrive/CowExtractionFromVideos/raw_single_cow_videos'
# ---------------------------

# Smoothing
SMOOTHING_ALPHA = 0.2

# Model Settings
# Colab'da güçlü GPU olduğu için 'x' (extra large) model kullanılabilir.
YOLO_MODEL_NAME = 'yolov8x-seg.pt' 

# Processing
BORDER_MARGIN = 5
CONFIDENCE_THRESHOLD = 0.75
TARGET_CLASS_ID = 19
BACKGROUND_COLOR = (0, 0, 0)

# Masking Settings
MASK_METHOD = 'soft'  # 'soft' veya 'binary'
MASK_BLUR_KERNEL_SIZE = (15, 15)
MASK_DILATION_ITERATIONS = 2

VIDEO_EXT = '.mp4'
OUTPUT_RESOLUTION = (640, 640)
MIN_TRACK_DURATION_SEC = 4.0
CROP_PADDING = 30 # Extra pixels

os.makedirs(OUTPUT_VIDEOS_DIR, exist_ok=True)
"""

# Yeni ayarları config/settings.py dosyasına yazıyoruz
with open('config/settings.py', 'w') as f:
    f.write(settings_content)

print("Ayarlar Colab için güncellendi (yolov8x-seg modeli seçildi, yollar ayarlandı).")

In [ ]:
# 6. İşlemi Başlat
!python main.py

In [ ]:
# 7. Sonuçları Görüntüleme (Örnek bir video oynat)
import glob
from IPython.display import HTML
from base64 import b64encode

output_dir = '/content/drive/MyDrive/CowExtractionFromVideos/output_cows'
videos = glob.glob(f"{output_dir}/*.mp4")

if videos:
    video_path = videos[0]
    print(f"Oynatılan video: {video_path}")
    
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    
    display(HTML(f"""
    <video width=400 controls>
          <source src="{data_url}" type="video/mp4">
    </video>
    """))
else:
    print("Henüz bir çıktı videosu bulunamadı.")